In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import os
import os
import sys
import pandas as pd
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
import plotly.express as px

In [2]:
dataset_path = r'C:\Users\TPWODL\New folder_Content\AutonomousDataAnalystAgent\data\raw\x_data.xlsx'

In [3]:
df = pd.read_excel(r'C:\Users\TPWODL\New folder_Content\AutonomousDataAnalystAgent\data\raw\x_data.xlsx')


In [11]:
df.shape

(32873, 34)

In [6]:
import pandas as pd

def agging_open_close_pivot_dict(dataset_path: str) -> dict:
    """
    Reads an Excel file, creates pivot tables of complaint type vs age bucket
    (for open and closed complaints), adds grand totals (row + column),
    merges them, and returns the result as a dictionary.
    
    Parameters
    ----------
    dataset_path : str
        Path to the Excel dataset.
    
    Returns
    -------
    dict
        Dictionary representation of the pivot table with totals.
    """
    # Load dataset
    old_df = pd.read_excel(dataset_path)
    
    # Filter for open/closed complaints
    open_df = old_df[old_df['CLOSED/OPEN'].str.lower().str.strip() == 'open']
    close_df = old_df[old_df['CLOSED/OPEN'].str.lower().str.strip() == 'close']

    # Ensure DATE column is datetime
    open_df['DATE'] = pd.to_datetime(open_df['DATE'])
    close_df['DATE'] = pd.to_datetime(close_df['DATE'])

    # Calculate age in days
    today = pd.Timestamp.today()
    open_df['Age_Days'] = (today - open_df['DATE']).dt.days
    close_df['Age_Days'] = (today - close_df['DATE']).dt.days

    # Define age buckets
    bins = [0, 15, 30, 60, 90, 180, float('inf')]
    labels = ['<15Days', '16-30Days', '31-60Days', '61-90Days', '91-180Days', '>180Days']
    open_df['Age_Bucket'] = pd.cut(open_df['Age_Days'], bins=bins, labels=labels, right=True, include_lowest=True)
    close_df['Age_Bucket'] = pd.cut(close_df['Age_Days'], bins=bins, labels=labels, right=True, include_lowest=True)    

    # Pivot tables
    pivot_open_data = pd.pivot_table(
        open_df,
        values='CLOSED/OPEN',
        index='COMPLAINT TYPE',
        columns='Age_Bucket',
        aggfunc='count',
        fill_value=0
    )

    pivot_close_data = pd.pivot_table(
        close_df,
        values='CLOSED/OPEN',
        index='COMPLAINT TYPE',
        columns='Age_Bucket',
        aggfunc='count',
        fill_value=0
    )
    
    # Reorder columns and ensure all labels exist (add missing columns with 0s)
    for label in labels:
        if label not in pivot_open_data.columns:
            pivot_open_data[label] = 0
        if label not in pivot_close_data.columns:
            pivot_close_data[label] = 0
    
    pivot_open_data = pivot_open_data[labels]
    pivot_close_data = pivot_close_data[labels]

    # Add Grand Total column
    pivot_open_data['Grand_Total'] = pivot_open_data.sum(axis=1)
    pivot_close_data['Grand_Total'] = pivot_close_data.sum(axis=1)

    # Add Grand Total row
    pivot_open_data.loc['Grand_Total'] = pivot_open_data.sum(axis=0)
    pivot_close_data.loc['Grand_Total'] = pivot_close_data.sum(axis=0)

    # Ensure integers
    pivot_open_data = pivot_open_data.astype(int)
    pivot_close_data = pivot_close_data.astype(int)

    # Reset index
    pivot_open_data = pivot_open_data.reset_index()
    pivot_close_data = pivot_close_data.reset_index()

    # Merge both tables (stack them vertically)
    merge_df = pd.concat([pivot_open_data, pivot_close_data], keys=['Open', 'Closed']).reset_index(level=0).rename(columns={'level_0':'Status'})

    # Convert to dictionary
    dict_pivot_data = merge_df.to_dict(orient='records')
    
    return dict_pivot_data

In [7]:
aging_df = agging_open_close_pivot_dict(dataset_path)

In [9]:
open_close_agging_df= pd.DataFrame(aging_df)

In [10]:
open_close_agging_df

,Status,COMPLAINT TYPE,<15Days,16-30Days,31-60Days,61-90Days,91-180Days,>180Days,Grand_Total
0,Open,Billing,0,0,1,0,0,0,1
1,Open,Civil works,5,11,18,14,54,46,148
2,Open,Low Voltage,0,0,1,0,0,0,1
3,Open,NSC,5,3,0,0,0,0,8
4,Open,Others,1,4,5,1,1,0,12
5,Open,Pole Shifting / Lt Sagging,1,4,5,5,25,32,72
6,Open,Safety,4,1,5,10,22,24,66
7,Open,Solar,1,0,0,0,0,0,1
8,Open,Transformer failure / NCC,2,0,0,0,0,0,2
9,Open,safety,0,0,0,0,0,1,1


In [ ]:
pivot_data = pd.pivot_table(
    df,
    index='COMPLAINT TYPE',
    columns=['DEPT', 'CLOSED/OPEN'],
    aggfunc='size',
    fill_value=0
)
pivot_data['Grand_Total'] = pivot_data.sum(axis=1)
pivot_data.loc['Grand_Total'] = pivot_data.sum(axis=0)
pivot_data = pivot_data.astype(int)  # Convert to Python int
pivot_data = pivot_data.reset_index()
dict_pivot_data = pivot_data.to_dict(orient='records')

In [13]:
pivot_data

DEPT                     COMPLAINT TYPE Commercial         O&M              \
CLOSED/OPEN                                 Closed Open Closed Open closed   
0                               Billing        818    1      0    0      0   
1                           Civil Works          0    0      1    0      0   
2                           Civil works          0    0   1750  148      2   
3                       Electrification          0    0    643    0      0   
4                           Low Voltage          0    0   1311    1      0   
5                              Metering        193    0      0    0      0   
6                       NO Power Supply          0    0      8    0      0   
7                                   NSC        768    8      0    0      0   
8                       No Power Supply          0    0   6686    0      0   
9                                Others          0    0    381    0      0   
10                              Payment        140    0      0    0      0   
11           Pole Shifting / Lt Sagging          0    0    972   72      0   
12                       Portal Problem        253    0      0    0      0   
13                         Power Outage          0    0  11443    0      1   
14                               Safety          0    0   1783   66      0   
15                                Solar         72    1      0    0      0   
16            Transformer failure / NCC          0    0    794    2      0   
17                              billing          1    0      0    0      0   
18                                  nsc          1    0      0    0      0   
19                               others          0    0      0    0      0   
20                               safety          0    0     44    1      0   
21                          Grand_Total       2246   10  25816  290      3   

DEPT         Other      Grand_Total  
CLOSED/OPEN Closed Open              
0                0    0         819  
1                0    0           1  
2                0    0        1900  
3                0    0         643  
4                0    0        1312  
5                0    0         193  
6                0    0           8  
7                0    0         776  
8                0    0        6686  
9             4423   12        4816  
10               0    0         140  
11               0    0        1044  
12               0    0         253  
13               0    0       11444  
14               2    0        1851  
15               0    0          73  
16               0    0         796  
17               0    0           1  
18               0    0           1  
19              71    0          71  
20               0    0          45  
21            4496   12       32873

In [15]:
pivot_data.shape

(22, 9)

In [16]:
dict_pivot_data = pivot_data.to_dict(orient='records')

In [17]:
dict_pivot_data

[{('COMPLAINT TYPE', ''): 'Billing',
  ('Commercial', 'Closed'): 818,
  ('Commercial', 'Open'): 1,
  ('O&M', 'Closed'): 0,
  ('O&M', 'Open'): 0,
  ('O&M', 'closed'): 0,
  ('Other', 'Closed'): 0,
  ('Other', 'Open'): 0,
  ('Grand_Total', ''): 819},
 {('COMPLAINT TYPE', ''): 'Civil Works',
  ('Commercial', 'Closed'): 0,
  ('Commercial', 'Open'): 0,
  ('O&M', 'Closed'): 1,
  ('O&M', 'Open'): 0,
  ('O&M', 'closed'): 0,
  ('Other', 'Closed'): 0,
  ('Other', 'Open'): 0,
  ('Grand_Total', ''): 1},
 {('COMPLAINT TYPE', ''): 'Civil works',
  ('Commercial', 'Closed'): 0,
  ('Commercial', 'Open'): 0,
  ('O&M', 'Closed'): 1750,
  ('O&M', 'Open'): 148,
  ('O&M', 'closed'): 2,
  ('Other', 'Closed'): 0,
  ('Other', 'Open'): 0,
  ('Grand_Total', ''): 1900},
 {('COMPLAINT TYPE', ''): 'Electrification',
  ('Commercial', 'Closed'): 0,
  ('Commercial', 'Open'): 0,
  ('O&M', 'Closed'): 643,
  ('O&M', 'Open'): 0,
  ('O&M', 'closed'): 0,
  ('Other', 'Closed'): 0,
  ('Other', 'Open'): 0,
  ('Grand_Total', '')

In [18]:
import pandas as pd

def open_close_complaint_report(dataset_path: str) -> dict:
    """
    Reads an Excel file, creates a pivot table of complaint type vs department & status,
    adds grand totals (row + column), and returns the result as a dictionary.

    Parameters
    ----------
    dataset_path : str
        Path to the Excel dataset.

    Returns
    -------
    dict
        Dictionary representation of the pivot table with totals.
    """
    # Load dataset
    df = pd.read_excel(dataset_path)

    # Create pivot table
    pivot_data = pd.pivot_table(
        df,
        index='COMPLAINT TYPE',
        columns=['DEPT', 'CLOSED/OPEN'],
        aggfunc='size',
        fill_value=0
    )

    # Add Grand Total column (row-wise sum)
    pivot_data['Grand_Total'] = pivot_data.sum(axis=1)

    # Add Grand Total row (column-wise sum)
    pivot_data.loc['Grand_Total'] = pivot_data.sum(axis=0)

    # Ensure integers
    pivot_data = pivot_data.astype(int)

    # Reset index to make 'COMPLAINT TYPE' a column
    pivot_data = pivot_data.reset_index()

    # Convert to dictionary
    dict_pivot_data = pivot_data.to_dict(orient='records')

    return dict_pivot_data


In [19]:
open_clode_data = open_close_complaint_report(dataset_path)

In [20]:
open_clode_data

[{('COMPLAINT TYPE', ''): 'Billing',
  ('Commercial', 'Closed'): 818,
  ('Commercial', 'Open'): 1,
  ('O&M', 'Closed'): 0,
  ('O&M', 'Open'): 0,
  ('O&M', 'closed'): 0,
  ('Other', 'Closed'): 0,
  ('Other', 'Open'): 0,
  ('Grand_Total', ''): 819},
 {('COMPLAINT TYPE', ''): 'Civil Works',
  ('Commercial', 'Closed'): 0,
  ('Commercial', 'Open'): 0,
  ('O&M', 'Closed'): 1,
  ('O&M', 'Open'): 0,
  ('O&M', 'closed'): 0,
  ('Other', 'Closed'): 0,
  ('Other', 'Open'): 0,
  ('Grand_Total', ''): 1},
 {('COMPLAINT TYPE', ''): 'Civil works',
  ('Commercial', 'Closed'): 0,
  ('Commercial', 'Open'): 0,
  ('O&M', 'Closed'): 1750,
  ('O&M', 'Open'): 148,
  ('O&M', 'closed'): 2,
  ('Other', 'Closed'): 0,
  ('Other', 'Open'): 0,
  ('Grand_Total', ''): 1900},
 {('COMPLAINT TYPE', ''): 'Electrification',
  ('Commercial', 'Closed'): 0,
  ('Commercial', 'Open'): 0,
  ('O&M', 'Closed'): 643,
  ('O&M', 'Open'): 0,
  ('O&M', 'closed'): 0,
  ('Other', 'Closed'): 0,
  ('Other', 'Open'): 0,
  ('Grand_Total', '')